In [3]:
import pandas as pd
import numpy as np

# Load data
portfolio = pd.read_csv("../data/client_portfolio.csv")
prices = pd.read_csv("../data/market_prices.csv")

# Merge portfolio with current prices
df = portfolio.merge(prices, on="asset_name", how="left")

# Calculate current value
df["current_value"] = df["units"] * df["current_price"]

# Profit / Loss
df["pnl"] = df["current_value"] - df["investment_amount"]

# Returns (%)
df["return_pct"] = (df["pnl"] / df["investment_amount"]) * 100

# Portfolio totals
total_invested = df["investment_amount"].sum()
total_current = df["current_value"].sum()
portfolio_return = ((total_current - total_invested) / total_invested) * 100

print("Total Invested:", total_invested)
print("Current Portfolio Value:", total_current)
print("Portfolio Return (%):", round(portfolio_return, 2))

df


Total Invested: 379700
Current Portfolio Value: 409500
Portfolio Return (%): 7.85


,client_id,asset_name,asset_class,units,avg_buy_price,investment_amount,risk_category,current_price,benchmark,current_value,pnl,return_pct
0,C001,RELIANCE,Equity,50,2400,120000,High,2650,NIFTY50,132500,12500,10.416667
1,C001,TCS,Equity,30,3500,105000,Medium,3800,NIFTY50,114000,9000,8.571429
2,C001,NIFTY50_ETF,Equity,40,180,7200,Medium,195,NIFTY50,7800,600,8.333333
3,C001,BOND_ETF,Debt,100,100,10000,Low,102,BOND_INDEX,10200,200,2.000000
4,C001,GOLD_ETF,Gold,25,5500,137500,Medium,5800,GOLD_INDEX,145000,7500,5.454545


In [4]:
# Asset allocation by class
allocation = df.groupby("asset_class")["investment_amount"].sum()
allocation_pct = allocation / allocation.sum() * 100

allocation_pct


asset_class
Debt       2.633658
Equity    61.153542
Gold      36.212800
Name: investment_amount, dtype: float64

In [5]:
target_allocation = {
    "Equity": 0.50,
    "Debt": 0.25,
    "Gold": 0.25
}

current_allocation = allocation_pct / 100

rebalance = pd.DataFrame({
    "Current Allocation": current_allocation,
    "Target Allocation": pd.Series(target_allocation)
})

rebalance["Action"] = rebalance["Target Allocation"] - rebalance["Current Allocation"]

rebalance

,Current Allocation,Target Allocation,Action
Debt,0.026337,0.25,0.223663
Equity,0.611535,0.50,-0.111535
Gold,0.362128,0.25,-0.112128
